#Files needed
* CNN_CIFAR10_net3.h5
* adv_de_cnn_cifar_part1_seed1
* adv_de_cnn_cifar_part2_seed1
* subset_cifar

# Import Libraries

In [1]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from scipy.stats import cauchy 
from scipy import stats
import operator
import random
from sklearn.metrics import accuracy_score
from skimage.measure import compare_ssim as ssim
import pickle
import cv2
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, Activation

#Set seeds
random.seed(0)
np.random.seed(0)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Get the data from pickle files

In [0]:

original = None
adversarial = None 
original_y = None
adversarial_y = None
adversarial_cnn_1 = None
adversarial_cnn_y_1  = None
adversarial_cnn_2 = None
adversarial_cnn_y_2  = None
times1 = None
times2 = None
nfes1 = None
nfes2 = None
psnrs1 = None
psnrs2 = None

with open('subset_cifar', 'rb') as f:
    original = pickle.load(f)
    adversarial= pickle.load(f)
    original_y = pickle.load(f)
    adversarial_y = pickle.load(f)

with open('adv_de_cnn_cifar_part1_seed1', 'rb') as f:
    adversarial_cnn_1 = pickle.load(f)
    times1 = pickle.load(f)
    nfes1 = pickle.load(f)
    psnrs1 = pickle.load(f)
    adversarial_cnn_y_1= pickle.load(f)

with open('adv_de_cnn_cifar_part2_seed1', 'rb') as f:
    adversarial_cnn_2 = pickle.load(f)
    times2 = pickle.load(f)
    nfes2 = pickle.load(f)
    psnrs2 = pickle.load(f)
    adversarial_cnn_y_2= pickle.load(f)

In [21]:
times = times1 + times2
nfes = nfes1 + nfes2
print(np.around(np.mean(times),3), "+-" ,np.around(np.std(times),3),"(" ,np.around(np.min(times),3), ",",
                                                                                        np.around(np.max(times),3),")")
print(np.around(np.mean(nfes),3), "+-" ,np.around(np.std(nfes),3),"(" ,np.around(np.min(nfes),3), ",",
                                                                                        np.around(np.max(nfes),3),")")

335.128 +- 105.796 ( 13.803 , 563.824 )
1237.526 +- 399.686 ( 50 , 2199 )


In [23]:
len(times)

133

In [0]:
adversarial_cnn_y = adversarial_cnn_y_1 + adversarial_cnn_y_2

In [0]:
adversarial_cnn = np.concatenate((adversarial_cnn_1[:91],adversarial_cnn_2[91:]),axis=0)

In [0]:
original.shape

(914, 32, 32, 1)

In [0]:
adversarial.shape

(133, 32, 32, 1)

In [0]:
adversarial_cnn.shape

(133, 32, 32, 1)

# Calculate PSNR and SSIM and Euclidean

In [0]:
psnrs_cnn = []
for i in range(adversarial.shape[0]):
  psnrs_cnn.append(cv2.PSNR(adversarial[i].reshape(32,32), adversarial_cnn[i].reshape(32,32)))

In [32]:
#double check GeeksForGeeks psnr
print(np.around(np.mean(psnrs_cnn),3), "+-" ,np.around(np.std(psnrs_cnn),3),"(" ,np.around(np.min(psnrs_cnn),3), ",",
                                                                                        np.around(np.max(psnrs_cnn),3),")")

22.16 +- 5.032 ( 10.549 , 31.507 )


In [33]:
ssims_cnn = []
for i in range(adversarial.shape[0]):
  ssims_cnn.append(ssim(adversarial[i].reshape(32,32), adversarial_cnn[i].reshape(32,32)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
print(np.around(np.mean(ssims_cnn),3), "+-" ,np.around(np.std(ssims_cnn),3),"(" ,np.around(np.min(ssims_cnn),3), ",",
                                                                                        np.around(np.max(ssims_cnn),3),")")

0.715 +- 0.186 ( 0.158 , 0.948 )


In [0]:
# https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
def euclidean(s):
    # pass s=a-b as argument to the function
    return np.linalg.norm(s)

In [0]:
eucs_cnn = []
for i in range(adversarial.shape[0]):
  eucs_cnn.append(euclidean(adversarial[i].reshape(32,32) -  adversarial_cnn[i].reshape(32,32)))

In [37]:
print(np.around(np.mean(eucs_cnn),3), "+-" ,np.around(np.std(eucs_cnn),3),"(" ,np.around(np.min(eucs_cnn),3), ",",
                                                                                        np.around(np.max(eucs_cnn),3),")")

760.248 +- 495.515 ( 216.944 , 2422.234 )


# Prepare Date

In [0]:
#Normalize images
original_sc = original        /255.0
adversarial_sc = adversarial     /255.0 
adversarial_cnn_sc = adversarial_cnn /255.0

# Predict Using CNN

In [0]:
from keras.models import load_model
model = load_model("CNN_CIFAR10_net3.h5")

In [0]:
#Get predictions
cnn_pred_original = model.predict(original_sc)
cnn_pred_adversarial = model.predict(adversarial_sc)
cnn_pred_adversarial_cnn = model.predict(adversarial_cnn_sc)

In [41]:
print("Accuracy on original: ",accuracy_score(original_y,np.argmax(cnn_pred_original,axis=1)))
print("Accuracy on adversarial: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial,axis=1)))
print("Accuracy on adversarial_cnn: ",accuracy_score(adversarial_y,np.argmax(cnn_pred_adversarial_cnn,axis=1)))

Accuracy on original:  0.824945295404814
Accuracy on adversarial:  0.8195488721804511
Accuracy on adversarial_cnn:  0.0


In [42]:
print("Accuracy on subset: ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial,axis=0),axis=1)))
print("Accuracy on subset with adversarial examples (CNN): ",accuracy_score(np.append(original_y,adversarial_y,axis=0),np.argmax(np.append(cnn_pred_original,cnn_pred_adversarial_cnn,axis=0),axis=1)))

Accuracy on subset:  0.8242597898758357
Accuracy on subset with adversarial examples (CNN):  0.720152817574021
